In [3]:
import sys
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# --- Funkcja budująca model ---
# Argument "hp" pochodzi od Keras Tuner i służy do wyboru hiperparametrów
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),  # Spłaszczenie obrazu 28x28 do 1D wektora długości 784
        # Warstwa gęsta z liczbą neuronów wybieraną przez Keras Tuner (od 32 do 512, co 32)
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        # Warstwa wyjściowa z 10 neuronami (po jednym na każdą cyfrę) i aktywacją softmax
        Dense(10, activation='softmax')
    ])

    # Kompilacja modelu z optymalizatorem Adam i współczynnikiem uczenia wybieranym przez Keras Tuner
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# --- Konfiguracja środowiska ---
sys.setrecursionlimit(100000)  # Zwiększenie limitu rekurencji (by uniknąć błędów z TensorFlow)
warnings.filterwarnings('ignore')  # Wyłączenie ostrzeżeń w konsoli
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Ograniczenie logów TensorFlow (ukrywa INFO i WARNING)

# --- Wczytanie i przetworzenie danych MNIST ---
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0  # Normalizacja pikseli do zakresu 0-1

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

# --- Konfiguracja Keras Tuner (Random Search) ---
tuner = kt.RandomSearch(
    build_model,                # Funkcja budująca model (z parametrami do strojenia)
    objective='val_accuracy',   # Co optymalizujemy: dokładność na danych walidacyjnych
    max_trials=10,              # Liczba różnych konfiguracji hiperparametrów do przetestowania
    executions_per_trial=2,     # Każda konfiguracja będzie trenowana 2 razy i wyniki będą uśrednione
    directory='my_dir',         # Gdzie zapisać dane z procesu strojenia
    project_name='intro_to_kt'  # Nazwa projektu (katalog roboczy)
)

# --- Podsumowanie przestrzeni hiperparametrów ---
tuner.search_space_summary()

# --- Uruchomienie strojenia hiperparametrów ---
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# --- Podsumowanie wyników strojenia ---
tuner.results_summary()

# --- Pobranie najlepszych hiperparametrów (z najlepszej konfiguracji) ---
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""

Najlepsza liczba neuronów w pierwszej warstwie Dense: {best_hps.get('units')}  
Najlepszy learning rate: {best_hps.get('learning_rate')}

""")

# --- Budowa i trening modelu z najlepszymi hiperparametrami ---
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)  # Trening z walidacją

# --- Ewaluacja modelu na danych walidacyjnych ---
test_loss, test_acc = model.evaluate(x_val, y_val)
print(f'Dokładność na zbiorze walidacyjnym: {test_acc}')


Trial 10 Complete [00h 01m 14s]
val_accuracy: 0.9773999750614166

Best val_accuracy So Far: 0.9801999926567078
Total elapsed time: 00h 16m 40s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
units: 448
learning_rate: 0.0005572109949030198
Score: 0.9801999926567078

Trial 05 summary
Hyperparameters:
units: 288
learning_rate: 0.0008019020731570195
Score: 0.9796999990940094

Trial 08 summary
Hyperparameters:
units: 288
learning_rate: 0.0017686030901453863
Score: 0.9785500168800354

Trial 06 summary
Hyperparameters:
units: 256
learning_rate: 0.0015516309595003499
Score: 0.9783500134944916

Trial 01 summary
Hyperparameters:
units: 384
learning_rate: 0.0018808082162961115
Score: 0.9782999753952026

Trial 00 summary
Hyperparameters:
units: 256
learning_rate: 0.0009973637481625263
Score: 0.9779999852180481

Trial 09 summary
Hyperparameters:
units: 320
learning_rate: 0.0013257717103432026
Sco